In [1]:
from ipywidgets import HBox,VBox,Layout,Box,Label,Image
from ASmartis import * 
import numpy as np
from Colors import Color

class KWSR_Diagramm(ASmartis):    
    
    def __init__(self,parameterList):
        self.width = 360
        self.height = 600
        self.maxTime = 10
        super().__init__(parameterList)
        #width = parameterList[0]
        #height = parameterList[1]
        self.stopSendInformation()
    
    def createButtons(self):
        self.buttonsObject = Buttons(self)
        
        for i in range(6):            
            self.buttonsObject.newTogglePlayPause("b_play_"+str(i),30,30)        
            self.buttonsObject.newFloatSlider("s_time_"+str(i), 0,self.maxTime,0,'',newOrientation='horizontal',newReadout=False)           
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
        
    def createChart(self):
        
        self.createArrowText()
        
        self.layers = ["background","arrowsConstant","arrowsVariable","arrowRes"]        
        
        self.chartsDict={}
        for i in range(6):
            self.chartsDict["chart"+str(i)]=Chart(self.layers, self.width, self.height)            
            if i > 2:
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","lineColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","fillingColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","lineWidth",2)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","lineColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","fillingColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","lineWidth",2)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","lineColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","fillingColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","lineWidth",5)
            else:
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","lineColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","fillingColor",Color.I.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsConstant","lineWidth",2)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","lineColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","fillingColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowsVariable","lineWidth",2)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","lineColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","fillingColor",Color.U.value)
                self.chartsDict["chart"+str(i)].changeLayerLook("arrowRes","lineWidth",5)
                        
            self.drawBackground(i)    
                
    def visualize(self):
        self.visualize_series()
        self.visualize_parallel()
        
    def update(self): 
        for i in range(6):
            if self.buttonsStatesDict["b_play_"+str(i)]:
                if (self.buttonsStatesDict["s_time_"+str(i)]+self.updateTime*2)>=self.maxTime:
                    self.buttonsObject.changeState("b_play_"+str(i),False)
                    self.buttonsObject.changeState("s_time_"+str(i),0)
                else:
                    self.buttonsObject.changeState("s_time_"+str(i),self.buttonsStatesDict["s_time_"+str(i)]+self.updateTime*2)
            if (self.buttonsStatesDict["s_time_"+str(i)] != self.oldButtonsStatesDict["s_time_"+str(i)] and
                self.buttonsStatesDict["s_time_"+str(i)] != 0):
                self.calculateArrowLength(i,self.buttonsStatesDict["s_time_"+str(i)])   
                
    def reset(self):
        pass 
    
    def createArrowText(self):
        layers = ["U","U_R","U_L","U_C","I","I_R","I_L","I_C"]
        imageChart = Chart(layers, 60,60)
        for i in range(len(layers)):
            if i > 3:
                imageChart.changeLayerLook(layers[i],"lineColor",Color.I.value)
                imageChart.changeLayerLook(layers[i],"fillingColor",Color.I.value)
                imageChart.drawText(layers[i], 26, 20, "30px serif", "I")
                imageChart.drawText(layers[i], 22, 20, "30px sans-serif", "_")
            else:
                imageChart.changeLayerLook(layers[i],"lineColor",Color.U.value)
                imageChart.changeLayerLook(layers[i],"fillingColor",Color.U.value)
                imageChart.drawText(layers[i], 20, 20, "30px sans-serif", "U")
                imageChart.drawText(layers[i], 22, 20, "30px sans-serif", "_")
                
        imageChart.drawText("U_R", 40, 26, "20px sans-serif", "R")
        imageChart.drawText("U_C", 40, 26, "20px sans-serif", "C")
        imageChart.drawText("U_L", 40, 26, "20px sans-serif", "L")
        imageChart.drawText("I_R", 40, 26, "20px sans-serif", "R")
        imageChart.drawText("I_C", 40, 26, "20px sans-serif", "C")
        imageChart.drawText("I_L", 40, 26, "20px sans-serif", "L")
        
        imageChart.update("U","U_R","U_L","U_C","I","I_R","I_L","I_C")
        
        self.arrowTextDict = {}        
        self.arrowTextDict["U/I"] = [imageChart.getImage("U"),imageChart.getImage("I")]        
        self.arrowTextDict["R"] = [imageChart.getImage("U_R"),imageChart.getImage("I_R")]        
        self.arrowTextDict["L"] = [imageChart.getImage("U_L"),imageChart.getImage("I_L")]        
        self.arrowTextDict["C"] = [imageChart.getImage("U_C"),imageChart.getImage("I_C")]
        
    def drawBackground(self,circuitNb):
        
        currentChart = self.chartsDict["chart"+str(circuitNb)]
        currentChart.drawArrow("background", 50, self.height/2, self.width-20, self.height/2, 20, 16)
        currentChart.drawText("background", self.width-50, self.height/2-10, "italic 35px serif", "Re")
        currentChart.drawArrow("background", 50, self.height, 50, 20, 20, 16)
        currentChart.drawText("background", 70, 35, "italic 30px serif", "j•Im")    
        
        currentChart.drawArrow("arrowsConstant", 50, self.height/2+8, 160 , self.height/2+8, 16, 12)
        if circuitNb < 3 : 
            currentChart.drawImage("arrowsConstant",self.arrowTextDict["U/I"][1], 0,self.height/2+4)
        else: 
            currentChart.drawImage("arrowsConstant",self.arrowTextDict["U/I"][0], 0,self.height/2+4)
        
        currentChart.update("background","arrowsConstant")
        
        
    def calculateArrowLength(self,circuit,time):
        
        arrow_res = 0
        
        if circuit == 0:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10*(-1)
            arrow_C = 0
            arrow_res = np.heaviside(time-6, 1)
        if circuit == 1:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = 0
            arrow_C = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10
            arrow_res = np.heaviside(time-6, 1)
        if circuit == 2:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10*(-1)
            arrow_C = (self.width-10)/100*((time-6)*np.heaviside(time-6, 1)-(time-9)*np.heaviside(time-9, 1))*10*2
            arrow_res = np.heaviside(time-9, 1)
        
        
        if circuit == 3:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10
            arrow_C = 0
            arrow_res = np.heaviside(time-6, 1)
        if circuit == 4:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = 0
            arrow_C = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10*(-1)
            arrow_res = np.heaviside(time-6, 1)
        if circuit == 5:
            arrow_R = (self.width-10)/100*(time*np.heaviside(time, 1)-(time-3)*np.heaviside(time-3, 1))*10
            arrow_L = (self.width-10)/100*((time-3)*np.heaviside(time-3, 1)-(time-6)*np.heaviside(time-6, 1))*10
            arrow_C = (self.width-10)/100*((time-6)*np.heaviside(time-6, 1)-(time-9)*np.heaviside(time-9, 1))*10*(-2)   
            arrow_res = np.heaviside(time-9, 1)
    
        self.redraw(circuit,arrow_R*2,arrow_L*2,arrow_C*2,arrow_res)
        
        
    def redraw(self,circuitNb,arrow_R,arrow_L,arrow_C,arrow_res):
        
        #print(circuit,arrow_R,arrow_L,arrow_C,arrow_res)
        
        x_start = 50
        y_start = self.height/2
        x = x_start
        y = y_start
        x_offset = 10 #Damit die Pfeile nebeneinander sind
        
        if circuitNb > 2 :
            circuit = 1
        else:
            circuit = 0
                
        currentChart = self.chartsDict["chart"+str(circuitNb)]
        currentChart.clearLayer("arrowsVariable")
        currentChart.clearLayer("arrowRes")
        arrowResPossible = False
        
        if abs(arrow_R) > 1:
            currentChart.drawArrow("arrowsVariable", x, y, x + arrow_R, y, 16, 12)    
            x = x + arrow_R
            currentChart.drawImage("arrowsVariable",self.arrowTextDict["R"][circuit], x-60,y+10)
            arrowResPossible = True
            
            
        if abs(arrow_L) > 1:
            if abs(arrow_C) > 1 : 
                x = x+ x_offset  
            currentChart.drawArrow("arrowsVariable", x, y, x, y + arrow_L, 16, 12)             
            y = y + arrow_L          
            currentChart.drawImage("arrowsVariable",self.arrowTextDict["L"][circuit], x-10,y-20)
            if abs(arrow_C) > 1: 
                x = x- x_offset
            arrowResPossible = True
                
        if abs(arrow_C) > 1:            
            currentChart.drawArrow("arrowsVariable", x, y, x, y + arrow_C, 16, 12) 
            y = y + arrow_C
            currentChart.drawImage("arrowsVariable",self.arrowTextDict["C"][circuit], x-10,y-20)
            arrowResPossible = True
            
        if (arrow_res and arrowResPossible):
            currentChart.drawArrow("arrowRes", x_start, y_start, x, y , 20, 16) 
            currentChart.drawImage("arrowRes",self.arrowTextDict["U/I"][circuit], x-70,y-14)
                
        currentChart.update("arrowsVariable","arrowRes")

        
    def visualize_series(self):
        buttonsDict = self.buttonsObject.getButtons()
        chart0 = self.chartsDict["chart"+str(0)].getChart()
        chart1 = self.chartsDict["chart"+str(1)].getChart()
        chart2 = self.chartsDict["chart"+str(2)].getChart()
        
        controlBox0 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_0"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_0"]],layout=Layout(width="20%"))])
        
        controlBox1 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_1"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_1"]],layout=Layout(width="20%"))])
        
        controlBox2 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_2"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_2"]],layout=Layout(width="20%"))])
        
        display(HBox([
            VBox([],layout=Layout(width="10%")),
            VBox([chart0,controlBox0],layout=Layout(width="20%")),
            VBox([],layout=Layout(width="10%")),
            VBox([chart1,controlBox1],layout=Layout(width="20%")),
            VBox([],layout=Layout(width="10%")),
            VBox([chart2,controlBox2],layout=Layout(width="20%"))
        ]))
    
    def visualize_parallel(self):
        buttonsDict = self.buttonsObject.getButtons()
        chart3 = self.chartsDict["chart"+str(3)].getChart()
        chart4 = self.chartsDict["chart"+str(4)].getChart()
        chart5 = self.chartsDict["chart"+str(5)].getChart()
        
        controlBox3 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_3"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_3"]],layout=Layout(width="20%"))])
        
        controlBox4 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_4"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_4"]],layout=Layout(width="20%"))])
        
        controlBox5 = HBox([Box([Label('t')],layout=Layout(justify_content= 'flex-end',width="10%")),                        
                            Box([buttonsDict["s_time_5"]],layout=Layout(width="70%")),
                            Box([buttonsDict["b_play_5"]],layout=Layout(width="20%"))])
        
        display(HBox([
            VBox([],layout=Layout(width="10%")),
            VBox([chart3,controlBox3],layout=Layout(width="20%")),
            VBox([],layout=Layout(width="10%")),
            VBox([chart4,controlBox4],layout=Layout(width="20%")),
            VBox([],layout=Layout(width="10%")),
            VBox([chart5,controlBox5],layout=Layout(width="20%"))
        ]))
    def getChart(self,circuitNb):
        chart = self.chartsDict["chart"+str(circuitNb)].getChart()
        return(chart)
        

In [2]:
if __name__ == '__main__':
    e = KWSR_Diagramm(None)
    e.visualize()
    e.start()

In [13]:
from ipywidgets import HBox,VBox,Layout,Box,Label,Image
import numpy as np
from ASmartis import * 
from Colors import Color
class KomplexeWechselstromrechnung(ASmartis):
    
    def __init__(self,parameterList):
        self.time = 0
        self.arrow_res_max = 250
        super().__init__(parameterList)
        self.reset()
        self.updateTime = 0.1
        
    def createButtons(self):
        
        self.buttonsObject = Buttons(self)    
        
        dd_options = [("RL-Reihenschaltung","RRL"),("RC-Reihenschaltung","RRC"),("RLC-Reihenschwingkreis","RRLC"),
            ("RL-Parallelschaltung","PRL"),("RC-Parallelschaltung","PRC"),("RLC-Parallelschwingkreis","PRLC")]
        self.buttonsObject.newDropdown("dd_circuit",dd_options,"")
        self.buttonsObject.newReset("b_reset", 'maxHeight','maxWidth')
        self.buttonsObject.newFloatSlider("s_scale", -15,15,0, "Maßstab:",newReadout=False)
        self.buttonsObject.newCheckBox("cb_scale","Automatisch")
        
        #Widerstand Buttons
        self.buttonsObject.newIntSlider("s_resistor", 1,999,200, "R",newReadout=False)
        self.buttonsObject.newFloatTextbox("t_resistor", 999, "")
        self.buttonsObject.linkingButtons("s_resistor","t_resistor")
        self.buttonsObject.newDropdown("dd_resistor",[("Ohm",1), ("kOhm",1000),("mOhm",0.001)],"")
        
        #Spulen Buttons
        self.buttonsObject.newIntSlider("s_coil", 1,999,250, "L",newReadout=False)
        self.buttonsObject.newFloatTextbox("t_coil", 1, "")
        self.buttonsObject.linkingButtons("s_coil","t_coil")
        self.buttonsObject.newDropdown("dd_coil",[("µH",0.000001),("mH",0.001),("H",1)],"")
        
        #Kondensator Buttons
        self.buttonsObject.newIntSlider("s_capacitor", 1,999,10, "C",newReadout=False)
        self.buttonsObject.newFloatTextbox("t_capacitor", 1, "")
        self.buttonsObject.linkingButtons("s_capacitor","t_capacitor")
        self.buttonsObject.newDropdown("dd_capacitor",[("pF",0.000000000001), ("nF",0.000000001),("µF",0.000001),("mF",0.001)],"")
        
        #Frequenz Buttons
        self.buttonsObject.newIntSlider("s_frequence", 1,999,50, "f",newReadout=False)
        self.buttonsObject.newFloatTextbox("t_frequence", 1, "")
        self.buttonsObject.linkingButtons("s_frequence","t_frequence")
        self.buttonsObject.newDropdown("dd_frequence",[("Hz",1), ("kHz",1000),("MHz",1000000),("GHz",1000000000)],"")
        
        #Buttons für Strom/Spannung
        self.buttonsObject.newIntSlider("s_cur/vol", 1,999,230, "",newReadout=False)
        self.buttonsObject.newFloatTextbox("t_cur/vol", 1, "")
        self.buttonsObject.linkingButtons("s_cur/vol","t_cur/vol")
        
        #Labels zum anzeigen der Daten        
        self.labelValue_ZY_R = Label(value = "",width = "50%")
        self.labelValue_ZY_L = Label(value = "",width = "50%")
        self.labelValue_ZY_C = Label(value = "",width = "50%")
        self.labelValue_ZY = Label(value = "",width = "50%")
        
        self.labelValue_UI_R = Label(value = "",width = "50%")
        self.labelValue_UI_L = Label(value = "",width = "50%")
        self.labelValue_UI_C = Label(value = "",width = "50%")
        self.labelValue_UI = Label(value = "",width = "50%")
        
        self.labelValueOmega = Label(value = "",width = "50%")
        self.labelValue_Phi = Label(value = "",width = "50%")        
        
        self.createButtonsDynamicDiagramm()
        
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
    
    def createChart(self):
     
        self.chartCircuit = Chart(["circuit"], 332, 225) 
        
        self.KWSR_Diagramm = KWSR_Diagramm(None)           
        self.chartDiagram = Chart(["diagram"], 360, 600)
        
        self.createChartDynamicDiagramm()
        
    def visualize(self):          
        self.visualize1()   
        self.visualize2()
    
    def visualize1(self):
        buttonsDict = self.buttonsObject.getButtons()
        
        self.buttonsObject.changeLayout(name = "dd_circuit",width = "70%")
        self.buttonsObject.changeLayout(name = "b_reset",width = "30%")
        boxCircuit = HBox([buttonsDict["dd_circuit"],buttonsDict["b_reset"]])
        
        self.buttonsObject.changeLayout(name = "s_scale",width = "70%")
        self.buttonsObject.changeLayout(name = "cb_scale",width = "30%")
        boxScale = HBox([buttonsDict["s_scale"],buttonsDict["cb_scale"]])
        
        self.buttonsObject.changeLayout(name = "s_resistor",width = "70%")
        self.buttonsObject.changeLayout(name = "t_resistor",width = "12%")
        self.buttonsObject.changeLayout(name = "dd_resistor",width = "18%")        
        boxResistance =  HBox([buttonsDict["s_resistor"],buttonsDict["t_resistor"], buttonsDict["dd_resistor"]])
        
        self.buttonsObject.changeLayout(name = "s_coil",width = "70%")
        self.buttonsObject.changeLayout(name = "t_coil",width = "12%")
        self.buttonsObject.changeLayout(name = "dd_coil",width = "18%")
        boxCoil = HBox([buttonsDict["s_coil"],buttonsDict["t_coil"],buttonsDict["dd_coil"]])
        
        self.buttonsObject.changeLayout(name = "s_capacitor",width = "70%")
        self.buttonsObject.changeLayout(name = "t_capacitor",width = "12%")
        self.buttonsObject.changeLayout(name = "dd_capacitor",width = "18%")
        boxCapacitor = HBox([buttonsDict["s_capacitor"],buttonsDict["t_capacitor"],buttonsDict["dd_capacitor"]])
        
        self.buttonsObject.changeLayout(name = "s_frequence",width = "70%")
        self.buttonsObject.changeLayout(name = "t_frequence",width = "12%")
        self.buttonsObject.changeLayout(name = "dd_frequence",width = "18%")
        boxFrequence = HBox([buttonsDict["s_frequence"],buttonsDict["t_frequence"],buttonsDict["dd_frequence"]])
           
        boxButtons = VBox([boxCircuit,boxScale,boxResistance,boxCoil,boxCapacitor,boxFrequence],layout = Layout(width = "60%"))
       
        circuitBox = Box([self.chartCircuit.getChart()],layout = Layout(width = "40%",height = "40%"))
        
        display(HBox([circuitBox,boxButtons],layout = Layout(justify_content = "space-between")))
        
        #Errechnete Ausgabe
        labelList = self.createLabels()
        
        self.valueOutputDict = {}
        
        for i in range(len(labelList)):
            labelDict = labelList[i]
        
            self.buttonsObject.changeLayout(name = "s_cur/vol",width = "60%")
            self.buttonsObject.changeLayout(name = "t_cur/vol",width = "20%")
            boxCurVol = HBox([labelDict["cur/vol"],buttonsDict["s_cur/vol"],buttonsDict["t_cur/vol"],labelDict["cur/vol_unit"]],layout = Layout(justify_content = "center"))
            
            boxResistanceLabel = VBox([labelDict["res/con_r"],labelDict["res/con_l"],labelDict["res/con_c"],labelDict["res/con_res"],labelDict["phi"]],layout = Layout(width = "20%",align_items = "flex-end"))
            boxResistanceValue = VBox([self.labelValue_ZY_R,self.labelValue_ZY_L,self.labelValue_ZY_C,self.labelValue_ZY,self.labelValue_Phi],layout = Layout(width = "50%",align_items = "flex-start"))
            boxResistanceUnit = VBox([labelDict["res/con_unit"],labelDict["res/con_unit"],labelDict["res/con_unit"],labelDict["res/con_unit"],labelDict["phi_unit"]],layout = Layout(width = "30%",align_items = "flex-start"))
            
            boxResistance = HBox([boxResistanceLabel,boxResistanceValue,boxResistanceUnit],layout = Layout(width = "50%", justify_content = "center"))
            
            boxVolCurLabel = VBox([labelDict["vol/cur_r"],labelDict["vol/cur_l"],labelDict["vol/cur_c"],labelDict["vol/cur_res"],labelDict["omega"]],layout = Layout(width = "20%",align_items = "flex-end"))
            boxVolCurValue = VBox([self.labelValue_UI_R,self.labelValue_UI_L,self.labelValue_UI_C,self.labelValue_UI,self.labelValueOmega],layout = Layout(width = "50%",align_items = "flex-start"))
            boxVolCurUnit = VBox([labelDict["vol/cur_unit"],labelDict["vol/cur_unit"],labelDict["vol/cur_unit"],labelDict["vol/cur_unit"],labelDict["omega_unit"]],layout = Layout(width = "30%",align_items = "flex-start"))
            
            boxVolCur = HBox([boxVolCurLabel,boxVolCurValue,boxVolCurUnit],layout = Layout(width = "50%", justify_content = "center"))

            boxValues = HBox([boxVolCur,boxResistance])
            partOutputBox = VBox([boxCurVol,boxValues])
            
            if i:
                self.valueOutputDict["series"] = partOutputBox
            else:
                self.valueOutputDict["parallel"] = partOutputBox

        boxOutput=  VBox([self.valueOutputDict["series"],self.valueOutputDict["parallel"]],layout = Layout(width = "60%"))
        
        chartBox = Box([self.chartDiagram.getChart()],layout = Layout(width = "30%")) 
        display(HBox([chartBox,boxOutput],layout = Layout(justify_content = "space-between", align_items = "center")))
        
        self.changeImageCircuit()
        self.updateChartDiagram()
        self.changeOutput()
        
    def createLabels(self):        
        
        seriesStringDict = {}        
        parallelStringDict = {}
        
        seriesStringDict["cur/vol"] = Label(value = "$\\color{%s}{\\underline{I}}$" % "#e8082c")        
        parallelStringDict["cur/vol"] = Label(value = "$\\color{%s}{\\underline{U}}$" % "#3333cc")        
        
        seriesStringDict["res/con_r"] = Label(value = "$\\underline{Z}_{R}=$")        
        parallelStringDict["res/con_r"] = Label(value = "$\\underline{Y}_{R}=$") 
        seriesStringDict["res/con_l"] = Label(value = "$\\underline{Z}_{L}=$")        
        parallelStringDict["res/con_l"] = Label(value = "$\\underline{Y}_{L}=$") 
        seriesStringDict["res/con_c"] = Label(value = "$\\underline{Z}_{C}=$")        
        parallelStringDict["res/con_c"] = Label(value = "$\\underline{Y}_{C}=$") 
        seriesStringDict["res/con_res"] = Label(value = "$|\\underline{Z}|=$")        
        parallelStringDict["res/con_res"] = Label(value = "$|\\underline{Y}|=$") 
        
        parallelStringDict["phi"] = Label(value = r"$\varphi=$")
        seriesStringDict["phi"] = Label(value = r"$\varphi=$")
        
        seriesStringDict["vol/cur_r"] = Label(value = "$\\color{%s}{\\rightarrow \\underline{U}_{R}}=$" % "#3333cc")        
        parallelStringDict["vol/cur_r"] = Label(value = "$\\color{%s}{\\rightarrow \\underline{I}_{R}}=$" % "#e8082c") 
        seriesStringDict["vol/cur_l"] = Label(value = "$\\color{%s}{\\uparrow \\underline{U}_{L}}=$" % "#3333cc")        
        parallelStringDict["vol/cur_l"] = Label(value = "$\\color{%s}{\\downarrow \\underline{I}_{L}}=$" % "#e8082c") 
        seriesStringDict["vol/cur_c"] = Label(value = "$\\color{%s}{\\downarrow \\underline{U}_{C}}=$" % "#3333cc")        
        parallelStringDict["vol/cur_c"] = Label(value = "$\\color{%s}{\\uparrow \\underline{I}_{C}}=$" % "#e8082c") 
        seriesStringDict["vol/cur_res"] = Label(value = "$\\color{%s}{|\\underline{U}|}=$" % "#3333cc")        
        parallelStringDict["vol/cur_res"] = Label(value = "$\\color{%s}{|\\underline{I}|}=$" % "#e8082c") 
        
        parallelStringDict["omega"] = Label(value = "$\omega=$")
        seriesStringDict["omega"] = Label(value = "$\omega=$")
        
        seriesStringDict["cur/vol_unit"] = Label(value = "$A$")        
        parallelStringDict["cur/vol_unit"] = Label(value = "$V$")        
        seriesStringDict["res/con_unit"] = Label(value = "$\Omega$")        
        parallelStringDict["res/con_unit"] = Label(value = "$S$")
        seriesStringDict["vol/cur_unit"] = Label(value = "$V$")        
        parallelStringDict["vol/cur_unit"] = Label(value = "$A$")        
        
        seriesStringDict["phi_unit"] = Label(value = "$°$")
        parallelStringDict["phi_unit"] = Label(value = "$°$")
        parallelStringDict["omega_unit"] = Label(value = "$1/s$")
        seriesStringDict["omega_unit"] = Label(value = "$1/s$")
        
        return [parallelStringDict,seriesStringDict]
    
    def update(self): 
        
        #Herausfilter der Buttons für das Reale Zeigerdiagramm
        oldButtonsStatesDictTMP = self.oldButtonsStatesDict.copy()
        oldButtonsStatesDictTMP.pop("cb_phi")
        oldButtonsStatesDictTMP.pop("s_phi")
        oldButtonsStatesDictTMP.pop("s_alpha")
        oldButtonsStatesDictTMP.pop("b_play")
        
        buttonsStatesDictTMP = self.buttonsStatesDict.copy()
        buttonsStatesDictTMP.pop("cb_phi")
        buttonsStatesDictTMP.pop("s_phi")
        buttonsStatesDictTMP.pop("s_alpha")
        buttonsStatesDictTMP.pop("b_play")
        
        
        if oldButtonsStatesDictTMP != buttonsStatesDictTMP:
            if not self.buttonsStatesDict["b_reset"]:
                if self.oldButtonsStatesDict["dd_circuit"] != self.buttonsStatesDict["dd_circuit"]:
                    self.changeOutput()
                    self.changeImageCircuit()
            else:
                self.partlyReset()
            self.updateValues()
            self.updateChartDiagram()
            
        #Abfrage der Buttons des Dynamischen Verhaltens
        if self.buttonsStatesDict["b_play"]:
            newTime = self.buttonsStatesDict["s_alpha"]+1
            if newTime>360:
                self.buttonsObject.changeState("b_play",False)
                newTime = 0
            self.buttonsObject.changeState("s_alpha",newTime)
                
                
        if self.buttonsStatesDict["cb_phi"] != self.oldButtonsStatesDict["cb_phi"]:
            self.changeDynamicOutput()
        if self.buttonsStatesDict["s_phi"] != self.oldButtonsStatesDict["s_phi"]:
            self.updateCurve(120,self.buttonsStatesDict["s_phi"],"curveCurrent")
        if (self.buttonsStatesDict["s_alpha"] != self.oldButtonsStatesDict["s_alpha"] or 
            self.buttonsStatesDict["s_phi"] != self.oldButtonsStatesDict["s_phi"]):
            
            self.updateArrow(150,0,"arrowVoltage","lineVoltage")
            self.updateArrow(120,self.buttonsStatesDict["s_phi"],"arrowCurrent","lineCurrent")    
            self.chartDynamic.update("curveVoltage","curveCurrent","arrowCurrent","lineCurrent","arrowVoltage","lineVoltage")
    
    def changeImageCircuit(self):
        
        circuit = self.buttonsStatesDict["dd_circuit"]
        image = Image.from_file("img/" + circuit + ".png")     
        self.chartCircuit.clearLayer("circuit")
        self.chartCircuit.update("circuit")
        self.chartCircuit.drawImageDirect("circuit",image,0,0)
        
    def changeOutput(self):
        
        if ("P") in self.buttonsStatesDict["dd_circuit"]:
            self.valueOutputDict["series"].layout.visibility = 'hidden'
            self.valueOutputDict["parallel"].layout.visibility = 'visible'
        else:
            self.valueOutputDict["series"].layout.visibility = 'visible'
            self.valueOutputDict["parallel"].layout.visibility = 'hidden'
         
        if ("L") in self.buttonsStatesDict["dd_circuit"]:
            self.buttonsObject.disabled("s_coil",False)
            self.buttonsObject.disabled("t_coil",False)
            self.buttonsObject.disabled("dd_coil",False)
        else:
            self.buttonsObject.disabled("s_coil",True)
            self.buttonsObject.disabled("t_coil",True)
            self.buttonsObject.disabled("dd_coil",True)
        
        if ("C") in self.buttonsStatesDict["dd_circuit"]:
            self.buttonsObject.disabled("s_capacitor",False)
            self.buttonsObject.disabled("t_capacitor",False)
            self.buttonsObject.disabled("dd_capacitor",False)
        else:
            self.buttonsObject.disabled("s_capacitor",True)
            self.buttonsObject.disabled("t_capacitor",True)
            self.buttonsObject.disabled("dd_capacitor",True)
                
    def updateValues(self): 
        
        if self.buttonsStatesDict["dd_circuit"] == "PRL":
            ZY_R = 1/(self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"])
            ZY_L = 1/(2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_coil"]*self.buttonsStatesDict["dd_coil"])
            ZY_C = 0
            ZY = sqrt((ZY_R)**2 +  (ZY_C-ZY_L)**2)  
            
        elif self.buttonsStatesDict["dd_circuit"] == "PRC":
            ZY_R = 1/(self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"])
            ZY_L = 0
            ZY_C = 2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_capacitor"]*self.buttonsStatesDict["dd_capacitor"]
            ZY = sqrt((ZY_R)**2 +  (ZY_C-ZY_L)**2)
            
        elif self.buttonsStatesDict["dd_circuit"] == "PRLC":
            ZY_R = 1/(self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"])
            ZY_L = 1/(2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_coil"]*self.buttonsStatesDict["dd_coil"])
            ZY_C = 2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_capacitor"]*self.buttonsStatesDict["dd_capacitor"]
            ZY = sqrt((ZY_R)**2 +  (ZY_C-ZY_L)**2)
             
        elif self.buttonsStatesDict["dd_circuit"] == "RRL":
            ZY_R = self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"]
            ZY_C = 0
            ZY_L = 2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_coil"]*self.buttonsStatesDict["dd_coil"]
            ZY = sqrt((ZY_R)**2 +  (ZY_L-ZY_C)**2)
            
        elif self.buttonsStatesDict["dd_circuit"] == "RRC":
            ZY_R = self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"]
            ZY_C = 1/(2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_capacitor"]*self.buttonsStatesDict["dd_capacitor"])
            ZY_L = 0
            ZY = sqrt((ZY_R)**2 +  (ZY_L-ZY_C)**2)
            
        elif self.buttonsStatesDict["dd_circuit"] == "RRLC":
            ZY_R = self.buttonsStatesDict["s_resistor"]*self.buttonsStatesDict["dd_resistor"]
            ZY_C = 1/(2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_capacitor"]*self.buttonsStatesDict["dd_capacitor"])
            ZY_L = 2*np.pi*self.buttonsStatesDict["s_frequence"]*self.buttonsStatesDict["dd_frequence"]*self.buttonsStatesDict["s_coil"]*self.buttonsStatesDict["dd_coil"]
            ZY = sqrt((ZY_R)**2 +  (ZY_L-ZY_C)**2)
        else:
            print("Fehler in der Abfrage")
            
        UI_R = ZY_R*self.buttonsStatesDict["s_cur/vol"]
        UI_L = ZY_L*self.buttonsStatesDict["s_cur/vol"]
        UI_C = ZY_C*self.buttonsStatesDict["s_cur/vol"]
        
        if ("P") in self.buttonsStatesDict["dd_circuit"]:
            UI = sqrt((UI_R)**2 +  (UI_C-UI_L)**2)
            Phi = np.arctan((UI_C-UI_L)/UI_R) 
        else:
            UI = sqrt((UI_R)**2 +  (UI_L-UI_C)**2)
            Phi = np.arctan((UI_L-UI_C)/UI_R)
        
        omega = 2*np.pi*self.buttonsStatesDict["s_frequence"]
        
        self.labelValueOmega.value = (f"%s" %str(round(omega,13)))[:16]
        
        self.labelValue_ZY_R.value = (f"%s" %str(round(ZY_R,13)))[:16]
        self.labelValue_ZY_L.value = (f"%s" %str(round(ZY_L,13)))[:16]
        self.labelValue_ZY_C.value = (f"%s" %str(round(ZY_C,13)))[:16]
        self.labelValue_ZY.value = (f"%s" %str(round(ZY,13)))[:16]
        self.labelValue_Phi.value = (f"%s" %str(round((Phi* 180 / np.pi),13)))[:16]
        
        self.labelValue_UI_R.value = (f"%s" %str(round(UI_R,13)))[:16]
        self.labelValue_UI_L.value = (f"%s" %str(round(UI_L,13)))[:16]
        self.labelValue_UI_C.value = (f"%s" %str(round(UI_C,13)))[:16]
        self.labelValue_UI.value = (f"%s" %str(round(UI,13)))[:16]
                   
        if self.buttonsStatesDict["cb_phi"]:
            self.buttonsObject.changeState("s_phi",(Phi* 180 / np.pi))
    
    def updateChartDiagram(self):
        #Maßstab berücksichtigen
        UI = float(self.labelValue_UI.value)
        
        if self.buttonsStatesDict["cb_scale"]:
            corr = self.arrow_res_max/UI
            self.buttonsObject.changeState("s_scale",np.log10(UI))
        else:
            corr = self.arrow_res_max/(10**self.buttonsStatesDict["s_scale"])

        arrow_R = float(self.labelValue_UI_R.value)*corr
        arrow_L = float(self.labelValue_UI_L.value)*corr
        arrow_C = float(self.labelValue_UI_C.value)*corr
        arrow_res = True
        
        if ("P") in self.buttonsStatesDict["dd_circuit"]:
            circuitNb = 3
            arrow_C = -arrow_C
        else:
            circuitNb = 0   
            arrow_L = -arrow_L
        
        self.KWSR_Diagramm.redraw(circuitNb,arrow_R,arrow_L,arrow_C,arrow_res)
        self.chartDiagram.clearLayer("diagram")
        self.chartDiagram.drawImage("diagram",self.KWSR_Diagramm.getChart(circuitNb),0,0)
        self.chartDiagram.update("diagram")
        
        
    def reset(self):
        
        self.buttonsObject.changeState("dd_circuit","RRL")  
                    
        #Buttons des Realen Zeigerdiagramms
        self.buttonsObject.changeState("cb_phi",True)
        self.buttonsObject.changeState("b_play",False)
        self.buttonsObject.changeState("s_alpha",0)        

        self.partlyReset()
        
        
        self.updateValues()
        
        self.changeDynamicOutput()
            
        self.updateCurve(150,0,"curveVoltage")
        self.updateCurve(120,self.buttonsStatesDict["s_phi"],"curveCurrent")
        
        self.updateArrow(150,0,"arrowVoltage","lineVoltage")
        self.updateArrow(120,self.buttonsStatesDict["s_phi"],"arrowCurrent","lineCurrent")
        self.chartDynamic.update("curveVoltage","curveCurrent","arrowCurrent","lineCurrent","arrowVoltage","lineVoltage")
        
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
        
    def partlyReset(self):
                      
        self.buttonsObject.changeState("cb_scale",True)
        
        #Widerstand Buttons
        self.buttonsObject.changeState("s_resistor", 200)
        self.buttonsObject.changeState("dd_resistor",1)
        
        #Spulen Buttons
        self.buttonsObject.changeState("s_coil", 250)
        self.buttonsObject.changeState("dd_coil",10**-3)
        
        #Kondensator Buttons
        self.buttonsObject.changeState("s_capacitor",10)
        self.buttonsObject.changeState("dd_capacitor",10**-6)
        
        #Frequenz Buttons
        self.buttonsObject.changeState("s_frequence",50)
        self.buttonsObject.changeState("dd_frequence",1)
        
        #Buttons für Strom/Spannung
        self.buttonsObject.changeState("s_cur/vol", 230)
        
        self.buttonsObject.changeState("b_reset", False)
        
    
    def createButtonsDynamicDiagramm(self):
        self.buttonsObject.newIntSlider("s_phi", -90,90,21, r"$\varphi$")
        self.buttonsObject.newIntSlider("s_alpha", 0,360,0, r"$\alpha$")
        self.buttonsObject.newCheckBox("cb_phi",r'$\varphi$ aus realem Zeigerdiagramm')
        self.buttonsObject.newTogglePlayPause("b_play", 40,40)
    
    def createChartDynamicDiagramm(self):
        #Graph für das dynamische Verhalten
        self.heigtChartDynamic = 500
        self.widthChartDynamic = 1200
        layers = ["background","curveVoltage","curveCurrent","arrowVoltage","arrowCurrent","lineVoltage","lineCurrent"]
        self.chartDynamic = Chart(layers, self.widthChartDynamic, self.heigtChartDynamic)
        
        for layer in layers:
            self.chartDynamic.changeLayerLook(layer,"lineWidth",3)
        
        self.chartDynamic.changeLayerLook("curveVoltage","lineColor",Color.U.value)
        self.chartDynamic.changeLayerLook("curveCurrent","lineColor",Color.I.value) 
        
        self.chartDynamic.changeLayerLook("arrowVoltage","lineColor",Color.U.value)
        self.chartDynamic.changeLayerLook("arrowCurrent","lineColor",Color.I.value)
        self.chartDynamic.changeLayerLook("arrowVoltage","fillingColor",Color.U.value)
        self.chartDynamic.changeLayerLook("arrowCurrent","fillingColor",Color.I.value) 
        
        self.chartDynamic.changeLayerLook("lineVoltage","lineColor",Color.U.value)
        self.chartDynamic.changeLayerLook("lineCurrent","lineColor",Color.I.value)
        self.chartDynamic.changeLayerLook("lineVoltage","lineDash",[2,4])
        self.chartDynamic.changeLayerLook("lineCurrent","lineDash",[2,4])
        
        self.drawDynamicDiagrammBackground()
        
    def visualize2(self):
        buttonsDict = self.buttonsObject.getButtons()
        
        playBox = HBox([buttonsDict["b_play"],buttonsDict["s_alpha"]])
        phiBox = HBox([buttonsDict["s_phi"],buttonsDict["cb_phi"]])
        
        display(VBox([self.chartDynamic.getChart(),playBox,phiBox]))
    
    def drawDynamicDiagrammBackground(self):
        self.x_center = 20
        self.chartDynamic.drawArrow("background",self.x_center,self.heigtChartDynamic/2,760,self.heigtChartDynamic/2,20,12)
        self.chartDynamic.drawArrow("background",self.x_center,self.heigtChartDynamic-50,self.x_center,50,20,12)
        self.chartDynamic.drawText("background",750,self.heigtChartDynamic/2-20,"italic 30px serif", "ω•t")
        
        self.chartDynamic.drawArrow("background",800,self.heigtChartDynamic/2,self.widthChartDynamic,self.heigtChartDynamic/2,20,12)
        self.chartDynamic.drawArrow("background",1000,self.heigtChartDynamic-50,1000,50,20,12)
        self.chartDynamic.drawText("background",1020,50,"italic 30px serif", "j•Im")
        self.chartDynamic.drawText("background",self.widthChartDynamic-40,self.heigtChartDynamic/2-20,"italic 30px serif", "Re")
        
        self.chartDynamic.changeLayerLook("background","fillingColor",Color.U.value)
        self.chartDynamic.drawText("background",self.x_center+20,40,"italic 30px serif", "u(t)")
        self.chartDynamic.changeLayerLook("background","fillingColor",Color.I.value)
        self.chartDynamic.drawText("background",self.x_center+20,70,"italic 30px serif", "i(t)")        
        
        self.chartDynamic.update("background")
        
    def updateCurve(self,amplitude,phi,currentLayer):    
        points = []
        for x in range(360):
            y = np.sin((x+phi+180)/180*np.pi)*amplitude+self.heigtChartDynamic/2
            points.append((x*2+self.x_center,y))
        self.chartDynamic.clearLayer(currentLayer)
        self.chartDynamic.drawLines(currentLayer,points)  
        self.chartDynamic.drawCircle(currentLayer,1000,self.heigtChartDynamic/2,amplitude,False) 
        
    def updateArrow(self,amplitude,phi,arrowLayer,lineLayer):
        time = self.buttonsStatesDict["s_alpha"]
        x1 = time*2+self.x_center
        x2 = -np.cos((time+phi+180)/180*np.pi)*amplitude+1000
        y = np.sin((time+phi+180)/180*np.pi)*amplitude+self.heigtChartDynamic/2
        
        self.chartDynamic.clearLayer(arrowLayer)
        self.chartDynamic.clearLayer(lineLayer)
        self.chartDynamic.drawCircle(arrowLayer,x1,y,5)              
        self.chartDynamic.drawArrow(arrowLayer,1000,self.heigtChartDynamic/2,x2,y,20,12)
        self.chartDynamic.drawLine(lineLayer,x1,y,x2,y)
        
     
    def changeDynamicOutput(self):
        if self.buttonsStatesDict["cb_phi"]:
            self.buttonsObject.disabled("s_phi",True)
            self.buttonsObject.changeState("s_phi",float(self.labelValue_Phi.value))
        else:
            self.buttonsObject.disabled("s_phi",False)
            
    def Test(self):
        pass

In [16]:
if __name__ == '__main__':
    f = KomplexeWechselstromrechnung(None)
    f.visualize1()
    f.start()

In [17]:
if __name__ == '__main__':
    f.visualize2()